##### This notebook demonstrates a demo of how diabetes-regression model can be trained and deployed on truefoundry platform.

## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/truefoundry-examples/blob/main/end-to-end-examples/diabetes-regression/diabetes_regression.ipynb)

In [ ]:
!pip install -U servicefoundry
!pip install -U mlfoundry

In [2]:
import os
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [1]:
!git clone https://github.com/truefoundry/truefoundry-examples.git
%cd truefoundry-examples
%cd end-to-end-examples/diabetes-regression

Cloning into 'truefoundry-examples'...
remote: Enumerating objects: 1630, done.
remote: Counting objects: 100% (513/513), done.
remote: Compressing objects: 100% (263/263), done.
remote: Total 1630 (delta 263), reused 409 (delta 233), pack-reused 1117
Receiving objects: 100% (1630/1630), 64.44 MiB | 11.97 MiB/s, done.
Resolving deltas: 100% (841/841), done.
/home/jovyan/truefoundry-examples
/home/jovyan/truefoundry-examples/end-to-end-examples/diabetes-regression


### Workspace FQN
Once you run the cell below you will get a prompt to enter your workspace. <br>
* Step 1: Click on the link given in the prompt.
* Step 2: Identify the Workspace you want to deploy the application in. 
* Step 3: Copy the Workspace FQN <br>
![Copying Workspace FQN](https://files.readme.io/730fee2-Screenshot_2023-02-28_at_2.08.34_PM.png)
* Step 4: Paste the Workspace FQN in the prompt and press enter.

In [4]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass
WORKSPACE=getpass("Please enter you WORKSPACE_FQN (https://app.truefoundry.com/workspaces):")

Please enter you WORKSPACE_FQN (https://app.truefoundry.com/workspaces): ··························


### API KEY
Once you run the cell below you will get a prompt to enter your API KEY. <br>
* Step 1: Click on the link given in the prompt.
* Step 2: Click on Create New API Key button
* Step 3: Enter the name you want to give to the API KEY and then click on generate
* Step 4: Copy the API KEY, You can also download the API KEY for later use <br>
![Copying API KEY](https://files.readme.io/201c8aa-Screenshot_2023-02-28_at_2.28.17_PM.png)
* Step 5: Paste the API KEY in the prompt and press enter.

In [5]:
TFY_API_KEY = getpass("Please enter your API Key (https://app.truefoundry.com/settings):")

Please enter your API Key (https://app.truefoundry.com/settings): ········································


In [6]:
os.environ["TFY_API_KEY"] = TFY_API_KEY

In [7]:
ML_REPO_NAME = input("Please Enter the ML repo name. You can create a new ML Repo from dashboard in the ML Repo section (Or enter any unique name, we will create a ML Repo for you): ")

Please Enter the ML repo name. You can create a new ML Repo from dashboard in the ML Repo section (Or enter any unique name, we will create a ML Repo for you):  diabetes-regression


In [8]:
os.environ["ML_REPO_NAME"] = ML_REPO_NAME

# ⚡ Training a ML Model using Jobs
In this section we will deploy a training job that trains a SVM model on iris dataset

In [9]:
%cd train

/home/jovyan/truefoundry-examples/end-to-end-examples/diabetes-regression/train


### Training script


In [10]:
!pygmentize train.py

import argparse
import os
import matplotlib.pyplot as plt
import mlfoundry
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.svm import SVR
from sklearn.metrics import PredictionErrorDisplay

def train(kernel: str, n_quantiles: int):
    # load the dataset
    X, y = load_diabetes(as_frame=True, return_X_y=True)

    # NOTE:- you can pass these configurations via command line
    # arguments, config file, environment variables.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # initialize the model
    regressor = SVR(kernel=kernel)

    # fit the model
    model = TransformedTargetRegressor(
        regressor=regressor,
        transformer=QuantileTransformer(n_quantiles=n_quantiles, output_distribution="normal"),
    )
    model.fit(X_train, y_train)

    

### Requirements File

In [11]:
!pygmentize requirements.txt

pandas==1.3.5
scikit-learn==1.2.1
mlfoundry>=0.7.2,<0.8.0
matplotlib==3.2.2


### Deployment using truefoundry

In [ ]:
import logging
import os
import argparse
from servicefoundry import Build, Job, PythonBuild, Param

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

job = Job(
    name="diabetes-reg-train",
    image=Build(
        build_spec=PythonBuild(
            command="python train.py --kernel {{kernel}} --n_quantiles {{n_quantiles}}",
            requirements_path="requirements.txt",
        )
    ),
    params=[
            Param(name="n_quantiles", default='100'),
            Param(name="kernel", default='linear', description="kernel for svm"),
        ],
    env={
        "TFY_API_KEY": os.environ['TFY_API_KEY'],
        "ML_REPO_NAME": os.environ['ML_REPO_NAME']
    },
)

deployment = job.deploy(workspace_fqn=WORKSPACE)

### Triggering a job's run

In [ ]:
from servicefoundry import trigger_job

APPLICATION_FQN="demo-euwe1-production:demos:diabetes-reg-train"

# Use the `trigger_job()` function to trigger a job run with different kernel values
for kernel in ["linear", "rbf", "poly"]:
    trigger_job(application_fqn=APPLICATION_FQN, params = {"kernel": kernel})

# ☁ Deploy ML Model using a Service (FastAPI)
In this section we will deploy the model trained and logged by our training job as an API

### Writing a fastapi service

In [3]:
%cd ../service-deploy

/home/jovyan/truefoundry-examples/end-to-end-examples/diabetes-regression/service-deploy


In [4]:
!pygmentize main.py

import os
from typing import List
import uuid
from datetime import datetime

import mlfoundry as mlf
import pandas as pd
from pydantic import BaseModel
from fastapi import FastAPI

# instantiate the FastAPI application via `FastAPI`
app = FastAPI(docs_url="/")

# get the model from `Truefoundry Model Registry`
MODEL_VERSION_FQN = os.environ["MODEL_VERSION_FQN"]
client = mlf.get_client()
model = client.get_model(MODEL_VERSION_FQN).load()

# define pydantic classes for the Instance, Request and Response
class Instance(BaseModel):
    age: float
    sex: float
    bmi: float
    bp: float
    s1: float
    s2: float
    s3: float
    s4: float
    s5: float
    s6: float


class Request(BaseModel):
    instances: List[Instance]

class Response(BaseModel):
    predictions: List[float]

# defining a fastapi endpoint for predictions
@app.post("/predict", response_model=Response)
def predict(request: Request):
    features = request.dict()["instances"]

    # create a dataframe out of the fea

### Requirements file

In [6]:
!pygmentize requirements.txt

pandas==1.3.5
scikit-learn==1.2.1
mlfoundry>=0.7.2,<0.8.0
fastapi==0.81.0
uvicorn==0.18.3


### Deploying Model as a service

#### MODEL VERSION FQN
Once you run the cell below you will get a prompt to enter your MODEL VERSION FQN. <br>
* Step 1: Click on the link given in the prompt.
* Step 2: Click on the Project Name (e.g. diabetes-regression)
* Step 3: Click the run with the highest score.
* Step 4: Click on the Model's tab.
* Step 5: Copy the MODEL VERSION FQN<br>
![Copying MODEL VERSION FQN](https://files.readme.io/9f726f6-Screenshot_2023-03-14_at_2.39.27_PM.png)
* Step 6: Paste the MODEL VERSION FQN in the prompt and press enter.

In [7]:
MODEL_VERSION_FQN = input("Please enter the model version fqn: (https://app.truefoundry.com/mlfoundry)")

Please enter the model version fqn: (https://app.truefoundry.com/mlfoundry) model:truefoundry/truefoundry-demo/diabetes-regression/diabetes-regression:2


In [ ]:
import logging

from servicefoundry import Build, Service, PythonBuild, Resources

#Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

service = Service(
    name="diabetes-reg-fastapi",
    image=Build(
        # Specify the build_specifications using `PythonBuild` to define we need a python environment
        build_spec=PythonBuild(
            command="uvicorn main:app --port 8000 --host 0.0.0.0",
        )
    ),
    ports=[{"port": 8000, "host": "diabetes-reg-fastapi.demo1.truefoundry.com"}],
    # Specify the environment variables
    env={
        "TFY_API_KEY": TFY_API_KEY,
        "MODEL_VERSION_FQN": MODEL_VERSION_FQN
    },
    resources=Resources(cpu_limit=0.2, cpu_request=0.2, memory_request=600, memory_limit=600, ephemeral_storage_limit=200, ephemeral_storage_request=200),
)
# Deploy the service using the `.deploy()` method
deployment = service.deploy(workspace_fqn=WORKSPACE)

# ☁ Deploy a batch-prediction job
In this section we deploy a batch prediction job which will load the model, make the predictions and log the data in monitoring

### Script to load the model and make predictions on a batch

In [16]:
%cd ../batch-inference

/home/jovyan/truefoundry-examples/end-to-end-examples/diabetes-regression/batch-inference


In [17]:
!pygmentize main.py

import mlfoundry as mlf
import os
import uuid
from datetime  import datetime
from sklearn.datasets import load_diabetes

# get the model from `Truefoundry Model Registry`.
MODEL_VERSION_FQN = os.environ["MODEL_VERSION_FQN"]
client = mlf.get_client()
model_version = client.get_model(MODEL_VERSION_FQN)

model_schema = model_version.model_schema

# load the model
model = model_version.load()

# load the dataset
X, y = load_diabetes(as_frame=True, return_X_y=True)

# we are sampling a smaller version of the dataset for demonstration purposes
X_sampled = X.sample(n=6)
y_sampled = y[X_sampled.index]

# make predictions from the model
y_pred = model.predict(X_sampled)

# parse the predictions in `mlfoundry.Prediction` format
predictions = []
for i in range(6):
    row = X_sampled.iloc[i]

    # create a dictionary of features
    features_dict = {}
    for feature in model_schema.features:
        features_dict[feature.name] = float(row[feature.name])

    #append `mlf.Prediction` objects in 

### Requirements file

In [18]:
!pygmentize requirements.txt

scikit-learn==1.2.1
mlfoundry>=0.7.2,<0.8.0


### Deploying the batch prediction job as a cron job

In [ ]:
import logging
import os
from servicefoundry import Build, Job, PythonBuild, Schedule

#Setup logging
logging.basicConfig(level=logging.INFO)

#Now we need to define the properties of the Job, using the `Job` class
job = Job(
    # Specifies what the name/identifier of the job should be
    name="diabetes-reg-batch",
    # Now we need to define the how to build our code into a Docker image, using `Build` class 
    image=Build(
        # Specify the build_specifications using `PythonBuild` to define we need a python environment
        build_spec=PythonBuild(command="python main.py"),
    ),
    # Specify the environment variables
    env={
        "MODEL_VERSION_FQN": MODEL_VERSION_FQN,
        "TFY_API_KEY": TFY_API_KEY
    },
    # To setup a Cron Job (Repeating Job), we will use the `Schedule` class.
    # You will use the `Schedule` class to specify the schedule of the Cron job.
    # Inside `Schedule` class you can pass the schedule argument. It takes the value in [Cron Format](https://docs.truefoundry.com/docs/deploy-a-cron-job#understanding-the-cron-format)
    trigger=Schedule(schedule="*/10 * * * *"),
)
# Deploy the job using the `.deploy()` method.
job.deploy(workspace_fqn=WORKSPACE)
